# Kaggle Download


In [1]:
%%capture
!pip install kaggle

In [2]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c cat-in-the-dat

cat-in-the-dat.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
# %%capture
# ! unzip cat-in-the-dat.zip

# Imports



In [7]:
import numpy as np
import pandas as pd

In [8]:
data = pd.read_csv("train.csv").drop(columns=["id"])
data.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [9]:
cat_cols = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5',
            'nom_6', 'nom_7', 'nom_8', 'nom_9', 'ord_0', 'ord_1', 'ord_2', 'ord_3',  'ord_4',  'ord_5', 'day', 'month']
data.shape

(300000, 25)

In [10]:
{col: data[f"{col}"].nunique() for col in list(data.columns)}

{'id': 300000,
 'bin_0': 2,
 'bin_1': 2,
 'bin_2': 2,
 'bin_3': 2,
 'bin_4': 2,
 'nom_0': 3,
 'nom_1': 6,
 'nom_2': 6,
 'nom_3': 6,
 'nom_4': 4,
 'nom_5': 222,
 'nom_6': 522,
 'nom_7': 1220,
 'nom_8': 2215,
 'nom_9': 11981,
 'ord_0': 3,
 'ord_1': 5,
 'ord_2': 6,
 'ord_3': 15,
 'ord_4': 26,
 'ord_5': 192,
 'day': 7,
 'month': 12,
 'target': 2}

In [11]:
data_one_hot = pd.get_dummies(data, columns=cat_cols)
data_one_hot.shape

(300000, 16463)

In [12]:
data_one_hot.head()

,id,target,bin_0_0,bin_0_1,bin_1_0,bin_1_1,bin_2_0,bin_2_1,bin_3_F,bin_3_T,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,0,0,True,False,True,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,1,0,True,False,False,True,True,False,False,True,...,False,False,False,False,False,True,False,False,False,False
2,2,0,True,False,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,3,1,True,False,False,True,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,4,0,True,False,True,False,True,False,True,False,...,False,False,False,False,False,True,False,False,False,False


# Embedding mogel

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import time

In [38]:
class CatEmbedding(nn.Module):

    def __init__(self, input_dim: int, embed_dim: int):
        super(CatEmbedding, self).__init__()

        self.input_dim = input_dim
        self.embed_dim = embed_dim
        self.model = nn.Sequential(
            nn.Linear(in_features=self.input_dim, out_features=self.embed_dim),
            nn.Linear(in_features=self.embed_dim, out_features=self.input_dim),
        )

    def forward(self, x):
        pred = self.model(x)
        return(pred)

In [39]:
def fit(model, opt, loss_fun, X_train, batch):
    model.train()
    for i in range(X_train.shape[0] // batch):
        x_batch = X_train[i * batch: (i + 1) * batch]
        pred = model(x_batch)
        err = loss_fun(x_batch, pred)

        err.backward()
        opt.step()
        opt.zero_grad()

In [40]:
def eval(model, loss_fun, X_test, batch):
    model.eval()

    with torch.no_grad():
        for i in range(X_test.shape[0] // batch):
            x_batch = X_test[i * batch: (i + 1) * batch]
            pred = model(x_batch)
            err = loss_fun(x_batch, pred)


In [41]:
def train(model, epochs, optim, loss_fun, X_train, X_test, batch_size):
    for epoch in tqdm(range(epochs)):
        print(f"Epoch {i + 1}")
        fit(model, optim, loss_fun, X_train, batch_size)
        eval(model, loss_fun, X_test, batch_size)

    print("Train was done")

In [61]:
embed_model = CatEmbedding(data_one_hot.shape[1] - 2, 500)
embed_model.parameters()

<generator object Module.parameters at 0x7bd641d7a500>

In [62]:
print(embed_model)

CatEmbedding(
  (model): Sequential(
    (0): Linear(in_features=16461, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=16461, bias=True)
  )
)


In [63]:
test = data_one_hot.iloc[:5, 2:]
test

KeyboardInterrupt: 

In [64]:
embed_model(torch.Tensor(test.values))

tensor([[ 0.0035,  0.0239, -0.0114,  ..., -0.0472, -0.0350,  0.0238],
        [-0.0024,  0.0417,  0.0044,  ..., -0.0394, -0.0351,  0.0382],
        [-0.0133,  0.0445,  0.0005,  ..., -0.0360, -0.0235,  0.0474],
        [ 0.0026,  0.0316, -0.0124,  ..., -0.0406, -0.0422,  0.0307],
        [-0.0085,  0.0337, -0.0075,  ..., -0.0275, -0.0260,  0.0330]],
       grad_fn=<AddmmBackward0>)

In [ ]:
optim = optim.AdamW(embed_model.parameters, lr=0.001)
loss = nn.BCE